In [1]:
%matplotlib inline
import importlib
import nn
importlib.reload(nn)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nn import NeuralNetwork
from sklearn.metrics import accuracy_score, f1_score

In [3]:
data_multimodal_train = pd.read_csv("dane_nn6/multimodal-sparse-training.csv")
X_multimodal_train = data_multimodal_train[['x']].values.reshape(-1, 1)
y_multimodal_train = data_multimodal_train['y'].values.reshape(-1, 1)

data_multimodal_test = pd.read_csv("dane_nn6/multimodal-sparse-test.csv")
X_multimodal_test = data_multimodal_test[['x']].values.reshape(-1, 1)
y_multimodal_test = data_multimodal_test['y'].values.reshape(-1, 1)

data_rings3_train = pd.read_csv("dane_nn6/rings3-balance-training.csv")
X_rings3_train = data_rings3_train[['x', 'y']].values.reshape(-1, 2)
y_rings3_train = data_rings3_train['c'].values.reshape(-1, 1)

data_rings3_test = pd.read_csv("dane_nn6/rings3-balance-test.csv")
X_rings3_test = data_rings3_test[['x', 'y']].values.reshape(-1, 2)
y_rings3_test = data_rings3_test['c'].values.reshape(-1, 1)

data_rings5_train = pd.read_csv("dane_nn6/rings5-sparse-training.csv")
X_rings5_train = data_rings5_train[['x', 'y']].values.reshape(-1, 2)
y_rings5_train = data_rings5_train['c'].values.reshape(-1, 1)

data_rings5_test = pd.read_csv("dane_nn6/rings5-sparse-test.csv")
X_rings5_test = data_rings5_test[['x', 'y']].values.reshape(-1, 2)
y_rings5_test = data_rings5_test['c'].values.reshape(-1, 1)

data_xor3_train = pd.read_csv("dane_nn6/xor3-balance-training.csv")
X_xor3_train = data_xor3_train[['x', 'y']].values.reshape(-1, 2)
y_xor3_train = data_xor3_train['c'].values.reshape(-1, 1)


In [4]:
def plot_classification_results(X_test, y_test, y_pred, title='Classification Results'):
    y_test = y_test.ravel()
    y_pred = y_pred.ravel()
    
    # Identify misclassified points
    misclassified = y_pred != y_test
    
    # Plot correctly classified points
    scatter = plt.scatter(X_test[~misclassified, 0], X_test[~misclassified, 1], 
                          c=y_test[~misclassified], cmap='viridis')
    
    # Plot misclassified points
    plt.scatter(X_test[misclassified, 0], X_test[misclassified, 1],
                color='red', marker='x', s=100, 
                label='Misclassified')
    
    plt.title(title)
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.legend()

In [5]:
nn_multimonn_mult1_linear = NeuralNetwork(X_multimodal_train, y_multimodal_train, [1, 96, 1],
                   activation_fun="linear", output_activation="linear", loss_fun="mse")

nn_multimodal = NeuralNetwork(X_multimodal_train, y_multimodal_train, [1, 32, 64, 32, 1],
                              activation_fun="tanh", output_activation="linear", loss_fun="mse")